In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

#### 1. Функция которая заходит на страницу и скролит вверх нужный div до конца

In [2]:
def get_page(page, driver):
    # Переходи на заданную страницу
    driver.get(page)

    # Подождать пока страница загрузится (можно уменьшить если все быстро)
    time.sleep(10)

    # Обращение по Селектору к элементу div, который надо прокрутить. Селектор можно скопировать из режима разработчика из браузера.
    element = driver.find_element(By.CSS_SELECTOR, '#card_holder > div.js-card-feed.card-holder__feed > div > div.notes-wrapper__scroller.custom-scroll')
    
    # Цикл прокрутки вверх до самого конца
    while True:
        # Прокручиваем вверх на максимальное возможное расстояние
        driver.execute_script("arguments[0].scrollTop = 0;", element)
        
        # Ждем некоторое время для загрузки нового контента
        time.sleep(3)
        
        # Проверяем, достигли ли мы самого начала содержимого элемента. Если достигли, то брейк.
        new_scroll_position = driver.execute_script("return arguments[0].scrollTop", element)
        if new_scroll_position == 0:
            break

    # Небольшая задержка для завершения анимации и догрузки
    time.sleep(8)

    # Получаем html полной страницы
    html_page = driver.page_source

    return html_page


#### 2. Функция которая цепляет все сообщения, которые находятся в div

In [3]:
def get_dff(html_code):
    # Цепляем html код с помощью BS
    soup = BeautifulSoup(html_code, 'html.parser')

    # Находим родительский элемент в котором лежат все сообщения
    div_elements = soup.find_all('div', class_='feed-note-wrapper feed-note-wrapper-undefined feed-note-wrapper-grouped-complex')

    # Лист для текста сообщений
    messeges_list = []
    # Булевое значение первого цикла, чтобы первое первое сообщение подхватить списком из двух. Так как первое всегда является техническим.
    first_iteration = True
    # Цикл который доходит до каждого элемента с сообщением и забирает текст
    for div in div_elements:
        for i in div.find_all('div','feed-note-wrapper feed-note-wrapper-amojo'):
            for content in i.find_all('div','feed-note__content'):
                for text in content.find_all('div','feed-note__body'):
                    mess = []
                    for str in text.find_all('div','feed-note__message_paragraph'):
                        mess.append(str.get_text(strip=True))
                    if first_iteration:
                        messeges_list.append(mess)
                        first_iteration = False
                    else:
                        mess1 = ' '.join(mess) 
                        messeges_list.append(mess1)


    # Лист для времени отправки сообщений
    time_list = []

    # Цикл доходит до каждого элемента с временем сообщения
    for div in div_elements:
        for i in div.find_all('div','feed-note-wrapper feed-note-wrapper-amojo'):
            for content in i.find_all('div','feed-note__content'):
                for head in content.find_all('div','feed-note__header-inner'):
                    for time in head.find_all('div','feed-note__date'):
                        time_list.append(time.get_text(strip=True))


    # Лист для имени, кто отправлял сообщения
    name_list = []

    # Цикл доходит до каждого элемента с именем отправителя
    for div in div_elements:
        for i in div.find_all('div','feed-note-wrapper feed-note-wrapper-amojo'):
            for content in i.find_all('div','feed-note__content'):
                for head in content.find_all('div','feed-note__header-inner'):
                    amojo_user_element = head.find('span', class_='feed-note__amojo-user')
                    name_list.append(amojo_user_element.get('title'))



    # # Лист для статусов сообщений
    # status_list = []

    # # Цикл дохдит для каждого элемента со статусом. Но есть косяк с тем, что указывается только статусы отправленные менеджером, поэтому этот список не используется.
    # for div in div_elements:
    #     for i in div.find_all('div','feed-note-wrapper feed-note-wrapper-amojo'):
    #         for content in i.find_all('div','feed-note__content'):
    #             # print(content)
    #             for head in content.find_all('div','feed-note__header-inner'):
    #                 # print(head)
    #                 for status in head.find_all('div','feed-note__delivery-status feed-note__checkmark_read'):
    #                     status_list.append(status.get_text(strip=True))


    # Формируется датафрейм с полученными списками 
    df1 = pd.DataFrame({
    'Name': name_list,
    'Message': messeges_list,
    'Time': time_list})

    return df1

#### 3. Лист с ссылками на сделки

In [4]:
list_of_pages = list(pd.read_excel('Проигранные сделки квал лиды WA (amo).xlsx')[111])

#### 4. Итоговое применение

In [ ]:
# Инициализируем драйвер браузера
driver = webdriver.Chrome()

# Загружаем первую страницу для захода под логином. Может быть любая.
driver.get('https://hellolondongatesorg.amocrm.ru/leads/detail/29712281')

# Ждем, пока все прогрузится. Как индикатор берем элемент с классом "custom_input__wrapper".
wait = WebDriverWait(driver, 10)
element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "custom_input__wrapper")))


# Находим поле ввода для логина по селектору "auth_form__field_wrapper"
login_field = driver.find_element(By.CSS_SELECTOR, ".auth_form__field_wrapper input[type='text']")

# Очищаем поле ввода перед вводом нового логина (если требуется)
login_field.clear()

# Вводим логин
login_field.send_keys("Логин")

# Находим поле ввода для пароля по селектору "auth_form__field_wrapper"
pass_field = driver.find_element(By.CSS_SELECTOR, ".auth_form__field_wrapper input[type='password']")

# Очищаем поле ввода перед вводом нового пароль (если требуется)
pass_field.clear()

# Вводим пароль
pass_field.send_keys("Пароль")

# Ждем чтобы все прогрузилось
driver.implicitly_wait(10)

# Нахождим кнопку для входя по селектору и кликаем
login_button = driver.find_element(By.CSS_SELECTOR, "button.auth_form__submit#auth_submit")
login_button.click()

# Опять ждем чтобы все прогрузилось
time.sleep(15)


# Создаем список, в котором будем хранить таблицу с диалогом каждой сделки
list_of_dialogs = []


# Циклом проходим по списоку заданных сделок. На каждую переходим по ссылке, парсим весь диалог и аппендим его в список list_of_dialogs
for link in list_of_pages:
    html_page = get_page(link, driver)
    list_of_dialogs.append(get_dff(html_page))